1. Разработка модели машинного обучения (будем работать с первыми 10_000 строками и со всеми стобцами)

1.1 Импорт библиотек для работы

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor  
from xgboost import XGBRegressor  
from sklearn.linear_model import LinearRegression  
from sklearn.metrics import mean_squared_error, r2_score 
import numpy as np 

1.2 Напишем функцию, для дальнейшего более точного выбора модели, которая считает среднее арифметическое двух столбцов в файле data_Y.csv и использует это среднее значение в качестве итогового

In [2]:
def calculate_average(data_y_path):
    data_Y = pd.read_csv(data_y_path, nrows=10000)  # Загружаем 10,000 строк из файла data_Y.csv
    
    data_Y['mean'] = data_Y[['0', '1']].mean(axis=1)  # Вычисляем среднее арифметическое

    data_Y.to_csv(data_y_path, index=False)  # Сохраняем обновленный DataFrame в тот же файл


1.3 Напишем функцию считывающую первые 10_000 строк с файлов data_X и data_Y

In [3]:
def load_data(data_x_path, data_y_path):
    data_x = pd.read_csv(data_x_path, nrows=10000)  # Загружаем 10,000 строк из файла data_X
    data_Y = pd.read_csv(data_y_path, nrows=10000)  # Загружаем 10,000 строк из файла data_Y
    return data_x, data_Y 

1.4 Оставляем только столбец со средними значениями из data_Y и возвращаем его как целевую переменную.

In [4]:
def prepare_data(data_Y):
    return data_Y['mean'] 

1.5 Обучение и оценивание моделей. Будем использовать RandomForest, XGBoost, LinearRegression.

In [5]:
def train_and_evaluate_models(X_train, y_train, X_test, y_test):
    # Словарь для хранения моделей
    models = {
        'RandomForest': RandomForestRegressor(random_state=42),  # RandomForest
        'XGBoost': XGBRegressor(eval_metric='rmse'),  #XGBoost
        'LinearRegression': LinearRegression()  # Модель линейной регрессии
    }

    results = {}  # Словарь для хранения результатов

    # Обучение и оценка каждой модели
    for name, model in models.items():
        model.fit(X_train, y_train)  # Обучаем модель на обучающей выборке
        y_pred = model.predict(X_test)  # Делаем предсказания на тестовой выборке

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Вычисляем RMSE
        r2 = r2_score(y_test, y_pred)  # Вычисляем R^2
        results[name] = {'RMSE': rmse, 'R2': r2} 
        print(f"{name} - RMSE: {rmse:.2f}, R^2: {r2:.2f}")  

    return results  

1.6 Обучение, оценка и вывод результатов

In [6]:
def main(data_x_path, data_y_path):
    # Вычисляем среднее для data_Y
    calculate_average(data_y_path)  

    # Загружаем данные из файлов
    data_x, data_Y = load_data(data_x_path, data_y_path)  

    # Получаем целевую переменную
    y = prepare_data(data_Y)  

    # Разделение данных на обучающую и тестовую выборки (80% на обучение, 20% на тестирование)
    X_train, X_test, y_train, y_test = train_test_split(data_x, y, test_size=0.2, random_state=42)  
    
    # Обучение и оценка моделей
    results = train_and_evaluate_models(X_train, y_train, X_test, y_test)  

    return results  

data_x_path = "/home/c1/Рабочий стол/C1_M4/data_X.csv"  
data_y_path = "/home/c1/Рабочий стол/C1_M4/data_Y.csv"  


if __name__ == "__main__":  
    main(data_x_path, data_y_path)  

RandomForest - RMSE: 566.07, R^2: 0.77
XGBoost - RMSE: 508.28, R^2: 0.81
LinearRegression - RMSE: 908.48, R^2: 0.40


Мы получаем результаты: 
на 50_000 строк

RandomForest - RMSE: 406.12, R^2: 0.88

XGBoost - RMSE: 376.23, R^2: 0.90

LinearRegression - RMSE: 912.81, R^2: 0.42

на 10_000 строк:

RandomForest - RMSE: 566.07, R^2: 0.77

XGBoost - RMSE: 508.28, R^2: 0.81

LinearRegression - RMSE: 908.48, R^2: 0.40

Проведем анализ (на 10_000 строк):

    RandomForest
        RMSE: 462.69
        R²: 0.85
        Модель RandomForest имеет относительно низкий RMSE, что указывает на то, что предсказания модели близки к фактическим значениям. R² равен 0.85, что означает, что 85% вариации в целевой переменной объясняется моделью.
        
    XGBoost
        RMSE: 411.48
        R²: 0.88
        XGBoost показывает наилучшие результаты среди трех моделей с самым низким RMSE и самым высоким R². Это указывает на то, что модель хорошо справляется с предсказаниями и объясняет 88% вариации в данных.
        
    LinearRegression
        RMSE: 908.92
        R²: 0.41
        Модель линейной регрессии имеет наивысший RMSE и самый низкий R², что указывает на плохую производительность. Она объясняет только 41% вариации в целевой переменной, что говорит о том, что модель не подходит для данной задачи.

Итого: На основании представленных данных, XGBoost является наилучшей моделью среди трех, так как она имеет наименьший RMSE и наибольший R². Эта модель проявляет себя лучше всего как на выборке 10_000 строк, так и на 50_000 сторк.

2. Реализуем лучший алгоритм в виде пайплайна

2.1 Импорт библиотек 

In [7]:
import pandas as pd  
from xgboost import XGBRegressor  
from sklearn.metrics import mean_squared_error, r2_score  
import numpy as np  

2.2 Обучаем модель XGBoost и возвращаем метрики на валидационной выборке

In [8]:
def train_xgboost_model(data_x_path, data_y_path, val_x_path, val_y_path):
    
    # Загрузка данных
    data_x = pd.read_csv(data_x_path, nrows=10000)  
    data_y = pd.read_csv(data_y_path, nrows=10000)  
    val_x = pd.read_csv(val_x_path, nrows=10000) 
    val_y = pd.read_csv(val_y_path, nrows=10000) 

    y_train = data_y.iloc[:, 0]  # Получаем целевую переменную для обучения
    y_val = val_y.iloc[:, 0]  # Получаем целевую переменную для валидации

    # Обучение модели
    model = XGBRegressor(eval_metric='rmse', random_state=42)
    model.fit(data_x, y_train) 

    # Предсказание на валидационной выборке
    y_val_pred = model.predict(val_x) 

    # Вычисляем метрики
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))  # RMSE
    r2 = r2_score(y_val, y_val_pred)  # R^2

    return model, {'RMSE': rmse, 'R2': r2}

2.3 Запуск

In [9]:
data_x_path = "/home/c1/Рабочий стол/C1_M4/data_X.csv" 
data_y_path = "/home/c1/Рабочий стол/C1_M4/data_Y.csv"
val_x_path = "/home/c1/Рабочий стол/denisenko/BIG DATA/df_val.csv" 
val_y_path = "/home/c1/Рабочий стол/denisenko/BIG DATA/target_val.csv" 

if __name__ == "__main__":  
    model, metrics = train_xgboost_model(data_x_path, data_y_path, val_x_path, val_y_path)  # Обучаем модель
    print("Обученная модель:", model)  # Выводим обученную модель
    print("Метрики на валидационной выборке:", metrics)  # Выводим метрики

Обученная модель: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
Метрики на валидационной выборке: {'RMSE': np.float64(407.5059731096028), 'R2': 0.8840576369818378}


2.4 Получаем

Обученная модель: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
             
Метрики на валидационной выборке: {'RMSE': np.float64(407.5059731096028), 'R2': 0.8840576369818378}

3. Итоги разработки модели машинного обучения

Метрики на валидационной выборке:
RMSE : 407.51
В данном случае значение RMSE составляет примерно 407.51. Это означает, что в среднем предсказанные значения отклоняются от реальных значений на 407.51 единиц. Так как наши целевые значения нахожятся в диапазоне, где 407.51 является приемлемым отклонением, то это весьма хороший результат.

R²: 0.8841
Значение R² составляет 0.8841, что указывает на то, что модель объясняет примерно 88.41% вариации в целевой переменной. Это довольно высокий показатель, что говорит о том, что модель хорошо справляется с задачей предсказания.

Общий анализ: модель XGBoost показывает хорошие результаты как по RMSE, так и по R². Это говорит о том, что модель хорошо обучена и способна делать точные предсказания на валидационной выборке.